In [1]:
import random
import time

### 환경 설정

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### 아웃카운트, 점수 시각화 함수

In [3]:
def visualize(outs, u_score, a_score):
    print(f"아웃카운트[", "🔴" * outs, "⚪" * (3-outs), "]", sep='', end=" ")
    print(f"👤 {u_score} : 🤖 {a_score}")

### 결과 확률 분포 함수

In [4]:
def decide_outcome():
    choices = ["1루타", "2루타", "3루타", "홈런", "플라이 아웃", "땅볼 아웃"]
    probs = [0.25, 0.10, 0.05, 0.05, 0.275, 0.275]
    return random.choices(choices, weights=probs, k=1)[0]

### 공격/수비 결과, 사투리 해설 함수

In [5]:
def generate_play(dialect, batter_item, glove_item, outcome, user_score, ai_score, player="사용자"):
    prompt = f"""
    ### 역할 ###
    너는 KBO에서 잘나가는 한국어 야구 중계 아나운서야.
    사용자가 선택한 지역 사투리({dialect})로 2줄 이내로 웃기고 생동감 있는 해설을 해줘.
    딱딱하게가 아니라 꼭 재미있는 개그와 밈을 섞어서 진행해야해.
    그리고 일반 해설가가 아니라 엄청난 {dialect} 사투리를 쓰는 해설가야.
    출력은 꼭 중계 멘트만 출력하고, 규칙 설명은 절대 출력 금지야.

    ### 경기 상황 ###
    플레이어: {player}
    타자 배트: {batter_item}, 수비 글러브: {glove_item}
    결과: {outcome}
    현재 점수: 👤 {user_score} : 🤖 {ai_score}

    ### 조건 ###
    - outcome 결과에 맞는 해설을 만들어줘
    - outcome과 해설은 반드시 일치해야해
    - 해설에 사투리 키워드 최소 2개 포함
    - 이모지 1~2개 사용

    [사투리 키워드 예시]
    {DIALECT}

    ### 출력 형식 ###
    commentary: [중계 멘트]
    """

    # 모델 로드
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=200
    )
    text = response.choices[0].message.content.strip()
    return text.replace("commentary:", "").strip()

### 게임 진행 함수

In [10]:
# 사투리 옵션
DIALECT = {
    "경상도": ["와이카노", "억수로", "아이가", "카이소", "아따", "이기 뭐꼬", "잡아뿔라"],
    "전라도": ["허벌나게", "거시기", "아따", "잉", "그라고", "혀붓다"],
    "충청도": ["그려", "그라지유", "어여", "~여유", "허다 보니께", "그라고말구"],
    "제주도": ["혼저옵서예", "하영", "간드락", "허멍", "고라보지 마씀", "민정신"]
}

# 배트 옵션
bat_options = ["나무빗자루", "플라스틱빗자루", "대걸래", "빨대", "바나나망치", "프라이팬", "우산", "젓가락", "해리포터 빗자루", "젤리스틱"]

# 글러브 옵션
glove_options = ["치킨 박스", "세숫대야", "비닐장갑", "목장갑", "풍선글러브", "아이스크림통", "화분", "빵틀", "털장갑"]

def play_game():
    # 인트로
    print("⚾ 세상에 이런 야구는 없었다 ⚾")
    dialect = ""
    while dialect not in DIALECT.keys():
        dialect = input(f"사용할 사투리를 선택하세요: {list(DIALECT.keys())} ")
    time.sleep(2)
    print(f"\n🧚해설핑: 안녕 나는 {dialect} 사투리로 게임을 중계할 해설핑이야~")
    time.sleep(2)
    print(f"🧚해설핑: 간단하게 룰 소개를 할게~")
    time.sleep(2)
    print(f"🧚해설핑: 여기서는 기본 야구 룰과 다르게 1루타만 쳐도 1점을 낼 수 있어~")
    time.sleep(2)
    print(f"🧚해설핑: 그러면 2루타는 2점, 3루타는 3점, 홈런은 4점이겠지~?")
    time.sleep(2)
    print(f"🧚해설핑: 좋은 도구들을 선택해서 AI를 꼭 이겨봐~")
    time.sleep(2)
    print("🧚해설핑: 그럼 이제 게임을 시작할게~\n")

    # 스코어 초기화
    user_score = 0
    ai_score = 0
    time.sleep(2)

    # 게임 시작
    print("<<<게임 start!>>>")

    # 이닝 설정
    for i in range(1, 2):
        print(f"🚩{i}이닝 시작")
        time.sleep(2)
        print("\n👤사용자 공격 차례!")
        time.sleep(2)

        # 아웃카운트 초기화
        outs = 0

        # 사용자 공격
        while outs < 3:
            batter_choices = random.sample(bat_options, 3)
            glove_choices = random.choice(glove_options)

            print(f"\n👉 🤖AI 선택 글러브: {glove_choices}")
            print(f"👉 👤선택 가능한 배트: {batter_choices}")

            batter_item = ""
            while batter_item not in batter_choices:
                batter_item = input("▶️ 선택할 배트를 입력하세요: ")
            glove_item = random.choice(glove_options)
            
            # 결과 선택
            outcome = decide_outcome()

            # 점수 적용
            if outcome in ["홈런"]:
                user_score += 4
            elif outcome in ["3루타"]:
                user_score += 3
            elif outcome in ["2루타"]:
                user_score += 2
            elif outcome in ["1루타"]:
                user_score += 1
            else:
                outs += 1

            # 해설
            commentary_text = generate_play(dialect, batter_item, glove_item, outcome, user_score, ai_score, player="사용자")
            time.sleep(2)
            print(f"\n[사용자 공격 결과👀] \n 👤 {batter_item} vs 🤖 {glove_item} → {outcome}\n")
            time.sleep(2)
            print(f"🧚해설핑: {commentary_text}\n")
            time.sleep(2)
            visualize(outs, user_score, ai_score)
            time.sleep(2)
            print("------------------------------------------------------------------------------")

        print("🔄️공수 교대! 🤖 AI 공격 차례")
        time.sleep(2)

        # 아웃카운트 초기화
        outs = 0

        # AI 공격
        while outs < 3:
            ai_batter = random.choice(bat_options)
            glove_choices = random.sample(glove_options, 3)

            print(f"\n👉 🤖AI 선택 배트: {ai_batter}")
            print(f"👉 👤 선택 가능한 글러브: {glove_choices}")

            glove_item = ""
            while glove_item not in glove_choices:
                glove_item = input("▶️ 선택할 글러브를 입력하세요: ")

            # 결과 선택
            outcome = decide_outcome()

            # 점수 적용
            if outcome in ["홈런"]:
                ai_score += 4
            elif outcome in ["3루타"]:
                ai_score += 3
            elif outcome in ["2루타"]:
                ai_score += 2
            elif outcome in ["1루타"]:
                ai_score += 1
            else:
                outs += 1

            # 해설
            commentary_text = generate_play(dialect, batter_item, glove_item,outcome, user_score, ai_score, player="AI")
            time.sleep(2)
            print(f"\n[AI 공격 결과👀] \n 🤖 {ai_batter} vs 👤 {glove_item} → {outcome}\n")
            time.sleep(2)
            print(f"🧚해설핑: {commentary_text}\n")
            time.sleep(2)
            visualize(outs, user_score, ai_score)
            time.sleep(2)
            print("------------------------------------------------------------------------------")

    # 게임 결과
    print("\n🏁 경기 종료!")
    if user_score > ai_score:
        print(f"경기 결과: 👤 {user_score} 대 🤖 {ai_score} \n 👤 사용자 승!!!!🎉")
    elif ai_score > user_score:
        print(f"경기 결과: 👤 {user_score} 대 🤖 {ai_score} \n 🤖 AI 승!!!!🎉")
    else:
        print(f"경기 결과: 👤 {user_score} 대 🤖 {ai_score} \n 🤝 무승부...")


### 게임 실행

In [11]:
play_game()

⚾ 세상에 이런 야구는 없었다 ⚾

🧚해설핑: 안녕 나는 제주도 사투리로 게임을 중계할 해설핑이야~
🧚해설핑: 간단하게 룰 소개를 할게~
🧚해설핑: 여기서는 기본 야구 룰과 다르게 1루타만 쳐도 1점을 낼 수 있어~
🧚해설핑: 그러면 2루타는 2점, 3루타는 3점, 홈런은 4점이겠지~?
🧚해설핑: 좋은 도구들을 선택해서 AI를 꼭 이겨봐~
🧚해설핑: 그럼 이제 게임을 시작할게~

<<<게임 start!>>>
🚩1이닝 시작

👤사용자 공격 차례!

👉 🤖AI 선택 글러브: 치킨 박스
👉 👤선택 가능한 배트: ['빨대', '해리포터 빗자루', '바나나망치']

[사용자 공격 결과👀] 
 👤 해리포터 빗자루 vs 🤖 화분 → 땅볼 아웃

🧚해설핑: 혼저옵서예! 해리포터 빗자루로 휘둘렀는데, 땅에 꽂히는 거 보소! 😂 화분으로는 잡아뿔라, 이기 뭐꼬! 점수는 여전히 0 대 0이여! 🌊

아웃카운트[🔴⚪⚪] 👤 0 : 🤖 0
------------------------------------------------------------------------------

👉 🤖AI 선택 글러브: 목장갑
👉 👤선택 가능한 배트: ['나무빗자루', '젓가락', '프라이팬']

[사용자 공격 결과👀] 
 👤 나무빗자루 vs 🤖 목장갑 → 홈런

🧚해설핑: 혼저옵서예! 이 보구마 뻥튀기 하고가, 나무빗자루로 공을 찌어버렸어! 홈런 뽕~~ 🎉 제주도에서 다들 고라보지 마씀, 이건 진짜 간드락이네! ⚾️

아웃카운트[🔴⚪⚪] 👤 4 : 🤖 0
------------------------------------------------------------------------------

👉 🤖AI 선택 글러브: 치킨 박스
👉 👤선택 가능한 배트: ['바나나망치', '대걸래', '빨대']

[사용자 공격 결과👀] 
 👤 빨대 vs 🤖 목장갑 → 홈런

🧚해설핑: 혼저옵서예! 빨대가 하영 날리면서 홈런 잡아뿔라! 🤤🤩 이게 바로 제주도 무치기, 목장갑 수비가 헛다리짚었네! 8:0으로 이기고 

---